# Exercise: Learning to recognise touch sounds
This exercise will look at recognising the audio registered by a piezo contact microphone on a mobile device when different parts of it are touched by a user. This is data from the **Stane** project ([Paper](http://www.dcs.gla.ac.uk/~rod/publications/MurWilHugQua08.pdf) and [video](http://www.dcs.gla.ac.uk/~rod/Videos/i_chi2.mov)), which used 3D printed surfaces to make super-cheap touch controllers.

<img src="imgs/stane_1.png" width="400px">
<img src="imgs/stane_2.png" width="400px">

The machine learning problem is simple: given a set of recordings of a user rubbing discrete touch zones on this 3D printed case, train a classifier which can distinguish which zone is being touched. This is in essence similar to speech recognition, but with a much simpler acoustic problem and no need to deal with language modeling.

We will use multi-class classification to distinguish the touch zones from the audio alone. We can assume a small number of discrete touch areas, and that there is no model governing how they might be touched (i.e. touches happen at random).



## A data processing pipeline
We need to develop a *pipeline* to process the data. There are several stages common to most supervised learning tasks:

1. Loading the original data (from files, databases etc.)
1. Pre-processing (removing outliers, resampling or interpolating, raw normalisation)
1. Feature extraction (transforming data into fixed length feature vectors)
1. Feature processing (offset removal and normalisation)
1. Data splitting (dividing into testing and training sections)
1. Classification (training the classifier)
1. Evaluation (testing the classifier performance)



In [ ]:
# standard imports
import numpy as np
import scipy.io.wavfile as wavfile
import scipy.signal as sig
import matplotlib.pyplot as plt
import seaborn


## Step 1: Loading the data
The first thing we need to do is to load the data. The data is in `datasets/stane/` and consists of four wave files from scratching four different surfaces, each 30 seconds in length, 44Khz, 16 bit PCM. 



In [2]:
%cd datasets
%ls

[Error 2] The system cannot find the file specified: u'datasets'
c:\work\summerschool2015
 Volume in drive C is OS
 Volume Serial Number is F2AC-1EA1

 Directory of c:\work\summerschool2015

10/06/2015  17:17    <DIR>          .
10/06/2015  17:17    <DIR>          ..
10/06/2015  08:56               759 .gitignore
10/06/2015  11:15    <DIR>          .ipynb_checkpoints
10/06/2015  17:17             1,344 CrashCourse_ExerciseAudio.ipynb
10/06/2015  15:57           298,980 CrashCourse_IntroIpython.ipynb
10/06/2015  16:27             4,291 CrashCourse_Lecture1.ipynb
10/06/2015  15:31             3,058 CrashCourse_Lecture2.ipynb
10/06/2015  10:47               577 CrashCourse_Lecture3.ipynb
10/06/2015  16:25             1,055 CrashCourse_Lecture4.ipynb
10/06/2015  11:14            10,127 CrashCourse_Resources.ipynb
10/06/2015  09:52    <DIR>          imgs
10/06/2015  08:56             1,096 LICENSE
10/06/2015  08:56                83 README.md
10/06/2015  15:57             4,081 SummerSchool

In [ ]:
# load each of the files into sound_files
sound_files = []
for texture in "1234":
    # load the wavefile
    sr, data = wavfile.load("scratch_%d"%texture)
    sound_files.append(data)
    
    

In [ ]:
## Step 2: Pre-processing
## Step 3: Feature extraction
## Step 4: Building a classifier
## Step 6: Evaluating performance
## Step 7: A better classifier
## Step 8: Better evaluation methods
## Step 9: Experiment